In [1]:
import pandas as pd
import numpy as np
from google.cloud import bigquery
from sklearn.metrics import pairwise_distances
from sklearn.neighbors import DistanceMetric
from scipy import sparse
from pandas.api.types import is_string_dtype, is_numeric_dtype
from sklearn.model_selection import train_test_split
import statsmodels.api as sm
import os

#### In the proceeding cell I import all of the features used to compute the simlarity matrix which include:
   - Spatial principal components computed at a zipcode level
   - ADP Embeddings for 2018
   - Census principal components computed on racial demographics, portion of income spent on rent, commute times, and educational attainment 
   
#### Potential features to add:
   - ADP percentage of jobs by sector, ie. % of retail jobs, % of healthcare jobs etc
   - EMSI job postings data
   - Zillow Single Family homes data, average price of singe family home in that zipcode 
   - ZRI Rental Rates 



In [61]:
os.environ["GOOGLE_APPLICATION_CREDENTIALS"] = "/Users/Dion/Documents/rf-research-1b6864db0b88.json"

query = """with spatial as (SELECT * FROM `rf-research.spatial_ai.all_zipcodes_pca_2020`),
                census as (SELECT * FROM `rf-research.acs.acs_5yr_zip_race_demo_pca`),
                adp as  (SELECT
                              em0.zip,
                              em0.year,
                              em0.embed embed0,
                              em1.embed embed1,
                              em2.embed embed2,
                              em3.embed embed3,
                              em4.embed embed4,
                              em5.embed embed5,
                              em6.embed embed6,
                              em7.embed embed7,
                              em8.embed embed8,
                              em9.embed embed9,
                              em10.embed embed10,
                              em11.embed embed11,
                              em12.embed embed12,
                              em13.embed embed13,
                              em14.embed embed14     
                              from `rf-research.embeddings.adp_homezip_zip_2008_2018` em0
                              INNER JOIN `rf-research.embeddings.adp_homezip_zip_2008_2018` em1
                              ON em0.zip = em1.zip AND em0.year = em1.year
                              INNER JOIN `rf-research.embeddings.adp_homezip_zip_2008_2018` em2
                              ON em0.zip = em2.zip AND em0.year = em2.year
                              INNER JOIN `rf-research.embeddings.adp_homezip_zip_2008_2018` em3
                              ON em0.zip = em3.zip AND em0.year = em3.year
                              INNER JOIN `rf-research.embeddings.adp_homezip_zip_2008_2018` em4
                              ON em0.zip = em4.zip AND em0.year = em4.year
                              INNER JOIN `rf-research.embeddings.adp_homezip_zip_2008_2018` em5
                              ON em0.zip = em5.zip AND em0.year = em5.year
                              INNER JOIN `rf-research.embeddings.adp_homezip_zip_2008_2018` em6
                              ON em0.zip = em6.zip AND em0.year = em6.year  
                              INNER JOIN `rf-research.embeddings.adp_homezip_zip_2008_2018` em7
                              ON em0.zip = em7.zip AND em0.year = em7.year   
                              INNER JOIN `rf-research.embeddings.adp_homezip_zip_2008_2018` em8
                              ON em0.zip = em8.zip AND em0.year = em8.year
                              INNER JOIN `rf-research.embeddings.adp_homezip_zip_2008_2018` em9
                              ON em0.zip = em9.zip AND em0.year = em9.year
                              INNER JOIN `rf-research.embeddings.adp_homezip_zip_2008_2018` em10
                              ON em0.zip = em10.zip AND em0.year = em10.year
                              INNER JOIN `rf-research.embeddings.adp_homezip_zip_2008_2018` em11
                              ON em0.zip = em11.zip AND em0.year = em11.year
                              INNER JOIN `rf-research.embeddings.adp_homezip_zip_2008_2018` em12
                              ON em0.zip = em12.zip AND em0.year = em12.year    
                              INNER JOIN `rf-research.embeddings.adp_homezip_zip_2008_2018` em13
                              ON em0.zip = em13.zip AND em0.year = em13.year
                              INNER JOIN `rf-research.embeddings.adp_homezip_zip_2008_2018` em14
                              ON em0.zip = em14.zip AND em0.year = em14.year       
                              WHERE em0.embedding_number = 0
                              AND em1.embedding_number = 1
                              AND em2.embedding_number = 2
                              AND em3.embedding_number = 3
                              AND em4.embedding_number = 4
                              AND em5.embedding_number = 5
                              AND em6.embedding_number = 6
                              AND em7.embedding_number = 7        
                              AND em8.embedding_number = 8                
                              AND em9.embedding_number = 9
                              AND em10.embedding_number = 10
                              AND em11.embedding_number = 11
                              AND em12.embedding_number = 12
                              AND em13.embedding_number = 13
                              AND em14.embedding_number = 14
                              AND em0.year = 2018) 
                  
                  
          select 
            s.*,
            c.census_pca_0,
            c.census_pca_1,
            c.census_pca_2,
            c.census_pca_3,
            c.census_pca_4,            
            c.census_pca_5,                        
            a.embed0,
            a.embed1,
            a.embed2,
            a.embed3,
            a.embed4,
            a.embed5,
            from 
          (select * from spatial) s
          inner join 
          (select * from census) c
          on s.zipcode = c.geo_id
          inner join 
          (select * from adp) a 
          on s.zipcode = a.zip"""



In [ ]:
client = bigquery.Client()
query_job = client.query(query)
df = query_job.result().to_dataframe()

- The following cell computes a euclidean distance matrix across all zipcodes. This matrix holds the similarity or distance score across every zipcode. I then find the index of the top 10 most similar markets. 

In [3]:
from sklearn.metrics.pairwise import euclidean_distances
index = df.set_index('zipcode').index.to_list()
euclid_index = euclidean_distances(df.set_index('zipcode').drop(columns=['DMA']), \
                                   df.set_index('zipcode').drop(columns=['DMA']))
df_euclid = pd.DataFrame(euclid_index, columns=index, index=index)

n = 10 
idx_euclid = np.argsort(df_euclid.values, 1)[:, 0:n]

- Find the index of Williamburg Brooklyn in original dataframe

In [7]:
df[df['zipcode'] == 11211]

,zipcode,DMA,pca_0,pca_1,pca_2,pca_3,pca_4,pca_5,pca_6,pca_7,...,census_pca_2,census_pca_3,census_pca_4,census_pca_5,embed0,embed1,embed2,embed3,embed4,embed5
2716,11211,New York (NY),-113.883542,-37.907282,77.760009,50.097242,-13.133189,-12.1012,-1.856425,-3.679566,...,22.60896,-1.358442,6.11628,-6.47129,0.789978,-0.274396,0.817554,0.474335,0.351988,-0.545739


- Display the dataframe which shows the top 10 most similar markets to Williambsurg.

- Results from the Williamsburg example below are all sensisble and accurate, East Village is the most similar neighborhood to Williamsburg. Followed by the Mission District in San Francisco and another Brooklyn neighborhood, Prospect Heights 

In [8]:
df.iloc[idx_euclid[2716]]

,zipcode,DMA,pca_0,pca_1,pca_2,pca_3,pca_4,pca_5,pca_6,pca_7,...,census_pca_2,census_pca_3,census_pca_4,census_pca_5,embed0,embed1,embed2,embed3,embed4,embed5
2716,11211,New York (NY),-113.883542,-37.907282,77.760009,50.097242,-13.133189,-12.101200,-1.856425,-3.679566,...,22.608960,-1.358442,6.116280,-6.471290,0.789978,-0.274396,0.817554,0.474335,0.351988,-0.545739
2450,10009,New York (NY),-113.892293,-40.987892,83.277113,45.570423,-16.380402,-0.580764,3.411143,15.061965,...,21.629419,-1.064418,3.169194,-6.589873,0.783947,-0.093243,0.809410,0.806996,0.551037,-0.652080
32971,94110,San Francisco (CA) - Oakland (CA) - San Jose (CA),-115.248768,-50.770940,69.902401,47.626935,-31.748450,-14.278086,-2.668369,4.065067,...,16.595051,1.063899,4.714256,-4.028000,0.883064,0.075601,0.605518,0.870827,-0.059690,0.166439
2742,11238,New York (NY),-129.178159,-45.993259,94.180195,46.806505,-5.978260,-23.983475,-18.143910,-6.852819,...,15.511862,-1.316825,7.512121,-2.348987,0.867314,-0.208553,1.059160,0.990095,0.683824,-0.616240
22080,60640,Chicago (IL),-128.487450,-40.220189,67.017547,58.733716,-13.782728,-9.219588,-7.514427,11.912905,...,14.208728,-0.279969,5.780806,-4.339890,0.911858,0.131784,1.014600,0.561650,0.159756,-0.700010
32004,90026,Los Angeles (CA),-119.770134,-32.163648,85.521848,35.676524,-15.016880,-7.880888,-11.175917,-0.545670,...,5.235031,0.281993,3.066854,-1.144749,0.796035,-0.137979,0.889127,-0.182107,0.074548,-0.085750
22060,60618,Chicago (IL),-106.916493,-52.496154,43.146763,48.151895,-18.965456,-11.905902,-8.544908,13.042768,...,8.942608,0.841010,8.565352,-0.584153,0.773976,0.314996,1.219456,0.343223,0.109220,-0.502457
2720,11215,New York (NY),-131.423766,-19.016103,60.398972,62.059659,7.200444,-33.520343,-10.963469,-6.236042,...,23.685748,-0.642657,10.467261,-1.146476,0.608282,0.089709,0.880991,0.915438,0.550896,-0.575502
2710,11205,New York (NY),-102.083316,-50.531829,79.984841,33.195673,3.467037,-19.560722,-14.123682,-2.442229,...,8.439216,-1.045095,0.815497,0.202212,0.445142,-0.044871,1.078193,0.458814,0.740502,-0.029665
2741,11237,New York (NY),-103.244058,-55.759964,91.415955,31.889969,-13.389349,-23.950025,-12.561300,15.143794,...,6.989046,-2.042150,5.583120,2.877421,0.831953,-0.726529,1.168384,0.138415,0.299978,-0.167733


- The output of the below cell answers the question, "What is the Williamsburg of Boston?"
- Our results show that the zipcode 02139, a neighborhood in Cambridge is the most similar to Williamsburg. 
- Having lived in Boston and looking over the inputs to the similarity score, this is an incrediblyy reasonsable and accurate response.
- This area of Boston is home to your post-college 24-30 year old, yuppy, higher earning demographic, commuters to downtown Boston, home to a plethora of Vegan restaraunts etc. 

In [63]:
df_zip_dma = df[['zipcode', 'DMA']]
df_willy = pd.merge(df_euclid.iloc[:,[2716]], df_zip_dma.set_index('zipcode'), left_index= True, right_index= True)
df_willy.columns = ['similarity_to_williamsburg', 'dma']
df_willy = df_willy.sort_values(by=['similarity_to_williamsburg'])
df_willy[df_willy['dma'] == 'Boston (MA) - Manchester (NH)'].head(1)

,similarity_to_williamsburg,dma
2139,65.038872,Boston (MA) - Manchester (NH)


- Moving forward I believe we should have a discussion around how we want this to persist in our data model and what the best way to present this to clients is.
- Should we provide this in a format where clients can explore their own similarities? Answer their own questions about what markets are similar to one another in a given city? 